# LESSON 2.6: Contrast Stretching (Normalization)
## Biomedical Image Processing Techniques

In this lesson:
- Min-Max normalization
- Linear contrast stretching
- Piecewise linear transformation
- Intensity-level slicing

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## 1. Why Contrast Stretching?

Many images don't use the full intensity range [0, 255].

**Contrast stretching** expands the range to improve visibility.

### Example:
- Original range: [50, 150]
- After stretching: [0, 255]

In [ ]:
# Create a low-contrast image
np.random.seed(42)
low_contrast = np.random.randint(80, 160, (200, 200), dtype=np.uint8)

# Add some structure
low_contrast[50:150, 50:150] = np.random.randint(100, 140, (100, 100), dtype=np.uint8)
low_contrast[80:120, 80:120] = np.random.randint(120, 150, (40, 40), dtype=np.uint8)

plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.imshow(low_contrast, cmap='gray', vmin=0, vmax=255)
plt.title('Low Contrast Image')
plt.colorbar()

plt.subplot(1, 2, 2)
plt.hist(low_contrast.ravel(), bins=256, range=(0, 256), color='steelblue')
plt.axvline(x=low_contrast.min(), color='r', linestyle='--', label=f'Min={low_contrast.min()}')
plt.axvline(x=low_contrast.max(), color='g', linestyle='--', label=f'Max={low_contrast.max()}')
plt.xlabel('Intensity')
plt.ylabel('Count')
plt.title('Histogram\n(Uses only part of [0, 255])')
plt.legend()
plt.xlim([0, 255])

plt.suptitle('Problem: Limited Dynamic Range', fontsize=14)
plt.tight_layout()
plt.show()

## 2. Min-Max Normalization

### Formula:
$$g(x,y) = \frac{f(x,y) - f_{min}}{f_{max} - f_{min}} \times 255$$

This maps:
- $f_{min} \rightarrow 0$
- $f_{max} \rightarrow 255$

In [ ]:
def min_max_stretch(image):
    """
    Apply min-max contrast stretching.
    
    Parameters:
    - image: input grayscale image (uint8)
    
    Returns:
    - stretched: contrast-stretched image (uint8)
    """
    img_float = image.astype(np.float32)
    
    f_min = img_float.min()
    f_max = img_float.max()
    
    # Avoid division by zero
    if f_max == f_min:
        return image
    
    # Apply formula
    stretched = (img_float - f_min) / (f_max - f_min) * 255
    
    return stretched.astype(np.uint8)

In [ ]:
# Apply min-max stretching
stretched = min_max_stretch(low_contrast)

plt.figure(figsize=(14, 5))

plt.subplot(1, 3, 1)
plt.imshow(low_contrast, cmap='gray', vmin=0, vmax=255)
plt.title(f'Original\nRange: [{low_contrast.min()}, {low_contrast.max()}]')
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(stretched, cmap='gray', vmin=0, vmax=255)
plt.title(f'After Stretching\nRange: [{stretched.min()}, {stretched.max()}]')
plt.colorbar()

plt.subplot(1, 3, 3)
plt.hist(low_contrast.ravel(), bins=256, range=(0, 256), alpha=0.5, label='Original', color='blue')
plt.hist(stretched.ravel(), bins=256, range=(0, 256), alpha=0.5, label='Stretched', color='red')
plt.xlabel('Intensity')
plt.ylabel('Count')
plt.title('Histogram Comparison')
plt.legend()
plt.xlim([0, 255])

plt.suptitle('Min-Max Contrast Stretching', fontsize=14)
plt.tight_layout()
plt.show()

## 3. Transformation Function Visualization

In [ ]:
# Visualize the transformation
f_min = low_contrast.min()
f_max = low_contrast.max()

x = np.arange(0, 256)
y = np.clip((x - f_min) / (f_max - f_min) * 255, 0, 255)

plt.figure(figsize=(8, 6))
plt.plot(x, y, 'b-', linewidth=2, label='Contrast Stretch')
plt.plot([0, 255], [0, 255], 'k--', alpha=0.5, label='Identity')

# Mark the input range
plt.axvline(x=f_min, color='r', linestyle=':', label=f'f_min = {f_min}')
plt.axvline(x=f_max, color='g', linestyle=':', label=f'f_max = {f_max}')

plt.xlabel('Input Intensity', fontsize=12)
plt.ylabel('Output Intensity', fontsize=12)
plt.title('Min-Max Stretching Transformation', fontsize=14)
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim([0, 255])
plt.ylim([0, 255])
plt.gca().set_aspect('equal')
plt.show()

## 4. Percentile-Based Stretching

Min-max stretching is sensitive to outliers (extreme values).

**Solution**: Use percentiles instead of min/max.

### Formula:
$$g = \frac{f - P_{low}}{P_{high} - P_{low}} \times 255$$

Where $P_{low}$ and $P_{high}$ are percentiles (e.g., 2nd and 98th).

In [ ]:
def percentile_stretch(image, low_percentile=2, high_percentile=98):
    """
    Apply percentile-based contrast stretching.
    
    Parameters:
    - image: input grayscale image (uint8)
    - low_percentile: lower percentile (default 2)
    - high_percentile: upper percentile (default 98)
    
    Returns:
    - stretched: contrast-stretched image (uint8)
    """
    img_float = image.astype(np.float32)
    
    p_low = np.percentile(img_float, low_percentile)
    p_high = np.percentile(img_float, high_percentile)
    
    if p_high == p_low:
        return image
    
    stretched = (img_float - p_low) / (p_high - p_low) * 255
    stretched = np.clip(stretched, 0, 255)
    
    return stretched.astype(np.uint8)

In [ ]:
# Create an image with outliers
np.random.seed(123)
with_outliers = np.random.randint(80, 160, (200, 200), dtype=np.uint8)

# Add bright outliers
with_outliers[10:15, 10:15] = 255
with_outliers[180:185, 180:185] = 255

# Add dark outliers
with_outliers[10:15, 180:185] = 0
with_outliers[180:185, 10:15] = 0

# Compare methods
minmax_result = min_max_stretch(with_outliers)
percentile_result = percentile_stretch(with_outliers, 2, 98)

plt.figure(figsize=(16, 5))

plt.subplot(1, 4, 1)
plt.imshow(with_outliers, cmap='gray', vmin=0, vmax=255)
plt.title('Original\n(with outliers)')
plt.colorbar()

plt.subplot(1, 4, 2)
plt.imshow(minmax_result, cmap='gray', vmin=0, vmax=255)
plt.title('Min-Max Stretch\n(affected by outliers)')
plt.colorbar()

plt.subplot(1, 4, 3)
plt.imshow(percentile_result, cmap='gray', vmin=0, vmax=255)
plt.title('Percentile Stretch (2-98%)\n(robust to outliers)')
plt.colorbar()

plt.subplot(1, 4, 4)
plt.hist(with_outliers.ravel(), bins=256, range=(0, 256), alpha=0.5, label='Original')
plt.hist(percentile_result.ravel(), bins=256, range=(0, 256), alpha=0.5, label='Percentile')
plt.xlabel('Intensity')
plt.legend()
plt.title('Histogram')

plt.suptitle('Min-Max vs Percentile Stretching', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Piecewise Linear Transformation

Define different linear functions for different intensity ranges.

### Example: Expand middle range, compress ends

```
        Output
    255 |      ____
        |     /
        |    /
        | __/
      0 |____________ Input
        0   r1  r2  255
```

In [ ]:
def piecewise_linear(image, r1, s1, r2, s2):
    """
    Apply piecewise linear transformation.
    
    Three segments:
    - [0, r1] -> [0, s1]
    - [r1, r2] -> [s1, s2]
    - [r2, 255] -> [s2, 255]
    
    Parameters:
    - image: input grayscale image
    - r1, s1: first breakpoint (input, output)
    - r2, s2: second breakpoint (input, output)
    
    Returns:
    - transformed image
    """
    # Create lookup table
    lut = np.zeros(256, dtype=np.uint8)
    
    for i in range(256):
        if i <= r1:
            # First segment
            if r1 == 0:
                lut[i] = 0
            else:
                lut[i] = int((s1 / r1) * i)
        elif i <= r2:
            # Middle segment
            lut[i] = int(s1 + ((s2 - s1) / (r2 - r1)) * (i - r1))
        else:
            # Last segment
            lut[i] = int(s2 + ((255 - s2) / (255 - r2)) * (i - r2))
    
    lut = np.clip(lut, 0, 255).astype(np.uint8)
    
    return lut[image], lut

In [ ]:
# Test piecewise linear transformation
np.random.seed(42)
test_img = np.random.randint(0, 256, (200, 200), dtype=np.uint8)

# Expand middle range (100-150) -> (50-200)
result, lut = piecewise_linear(test_img, r1=100, s1=50, r2=150, s2=200)

plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.imshow(test_img, cmap='gray', vmin=0, vmax=255)
plt.title('Original')
plt.colorbar()

plt.subplot(1, 3, 2)
plt.imshow(result, cmap='gray', vmin=0, vmax=255)
plt.title('Piecewise Linear\n(middle range expanded)')
plt.colorbar()

plt.subplot(1, 3, 3)
plt.plot(range(256), lut, 'b-', linewidth=2)
plt.plot([0, 255], [0, 255], 'k--', alpha=0.5)
plt.scatter([100, 150], [50, 200], color='red', s=100, zorder=5)
plt.xlabel('Input')
plt.ylabel('Output')
plt.title('Transformation Function\n(r1=100, s1=50, r2=150, s2=200)')
plt.grid(True, alpha=0.3)
plt.xlim([0, 255])
plt.ylim([0, 255])

plt.suptitle('Piecewise Linear Transformation', fontsize=14)
plt.tight_layout()
plt.show()

## 6. Intensity-Level Slicing

Highlight a specific intensity range.

### Two approaches:
1. **Without background**: Only show the range of interest
2. **With background**: Highlight range while preserving background

In [ ]:
def intensity_slice_no_bg(image, low, high, highlight_value=255):
    """
    Intensity slicing without background preservation.
    Pixels in [low, high] become highlight_value, others become 0.
    """
    result = np.zeros_like(image)
    mask = (image >= low) & (image <= high)
    result[mask] = highlight_value
    return result

def intensity_slice_with_bg(image, low, high, highlight_value=255):
    """
    Intensity slicing with background preservation.
    Pixels in [low, high] become highlight_value, others unchanged.
    """
    result = image.copy()
    mask = (image >= low) & (image <= high)
    result[mask] = highlight_value
    return result

In [ ]:
# Create test image
np.random.seed(789)
slice_test = np.random.randint(0, 256, (200, 200), dtype=np.uint8)

# Define range to highlight
low_val, high_val = 100, 150

no_bg = intensity_slice_no_bg(slice_test, low_val, high_val)
with_bg = intensity_slice_with_bg(slice_test, low_val, high_val)

plt.figure(figsize=(16, 5))

plt.subplot(1, 4, 1)
plt.imshow(slice_test, cmap='gray', vmin=0, vmax=255)
plt.title('Original')
plt.colorbar()

plt.subplot(1, 4, 2)
plt.imshow(no_bg, cmap='gray', vmin=0, vmax=255)
plt.title(f'Without Background\n[{low_val}, {high_val}] -> 255')
plt.colorbar()

plt.subplot(1, 4, 3)
plt.imshow(with_bg, cmap='gray', vmin=0, vmax=255)
plt.title(f'With Background\n[{low_val}, {high_val}] -> 255')
plt.colorbar()

plt.subplot(1, 4, 4)
x = np.arange(256)
y_no_bg = np.where((x >= low_val) & (x <= high_val), 255, 0)
y_with_bg = np.where((x >= low_val) & (x <= high_val), 255, x)
plt.plot(x, y_no_bg, 'b-', linewidth=2, label='Without BG')
plt.plot(x, y_with_bg, 'r-', linewidth=2, label='With BG')
plt.axvline(x=low_val, color='g', linestyle='--', alpha=0.5)
plt.axvline(x=high_val, color='g', linestyle='--', alpha=0.5)
plt.xlabel('Input')
plt.ylabel('Output')
plt.title('Transformation Functions')
plt.legend()
plt.xlim([0, 255])
plt.ylim([0, 260])

plt.suptitle('Intensity-Level Slicing', fontsize=14)
plt.tight_layout()
plt.show()

## 7. Medical Application: CT Window/Level Adjustment

In CT imaging, different tissues have different **Hounsfield Units (HU)**.

| Tissue | HU Range |
|--------|----------|
| Air | -1000 |
| Lung | -500 |
| Fat | -100 to -50 |
| Water | 0 |
| Soft tissue | +40 to +80 |
| Bone | +400 to +1000 |

**Window/Level** = contrast stretching for specific HU range

In [ ]:
def apply_ct_window(image, window_center, window_width):
    """
    Apply CT window/level (contrast stretching for CT).
    
    Parameters:
    - image: CT image (can be signed integers for HU)
    - window_center: center of the window (level)
    - window_width: width of the window
    
    Returns:
    - windowed image (uint8, 0-255)
    """
    img_float = image.astype(np.float32)
    
    # Calculate window bounds
    low = window_center - window_width / 2
    high = window_center + window_width / 2
    
    # Apply linear mapping within window
    windowed = (img_float - low) / (high - low) * 255
    windowed = np.clip(windowed, 0, 255)
    
    return windowed.astype(np.uint8)

In [ ]:
# Simulate a CT slice with different tissues
np.random.seed(456)

# Create CT image (values represent HU, shifted for uint8)
# We'll use 0-255 range but interpret as if it were HU
ct_image = np.zeros((256, 256), dtype=np.uint8)

# Background (air) - low values
ct_image[:, :] = 20

# Body outline
y, x = np.ogrid[:256, :256]
body_mask = ((x - 128)**2 / 90**2 + (y - 128)**2 / 100**2) <= 1
ct_image[body_mask] = np.random.randint(100, 130, ct_image[body_mask].shape)

# Lungs (dark)
lung_left = ((x - 90)**2 / 30**2 + (y - 120)**2 / 50**2) <= 1
lung_right = ((x - 166)**2 / 30**2 + (y - 120)**2 / 50**2) <= 1
ct_image[lung_left | lung_right] = np.random.randint(30, 50, ct_image[lung_left | lung_right].shape)

# Spine (bone - bright)
spine = ((x - 128)**2 + (y - 200)**2) <= 15**2
ct_image[spine] = np.random.randint(200, 240, ct_image[spine].shape)

# Ribs (bone)
for rib_y in [90, 120, 150]:
    rib_left = ((x - 60)**2 / 15**2 + (y - rib_y)**2 / 5**2) <= 1
    rib_right = ((x - 196)**2 / 15**2 + (y - rib_y)**2 / 5**2) <= 1
    ct_image[rib_left | rib_right] = 210

# Apply Gaussian smoothing
from scipy.ndimage import gaussian_filter
ct_image = gaussian_filter(ct_image, sigma=1).astype(np.uint8)

In [ ]:
# Different CT windows
# Lung window: center=50, width=100 (shows lung detail)
# Soft tissue window: center=120, width=80
# Bone window: center=200, width=150

lung_window = apply_ct_window(ct_image, 50, 100)
soft_window = apply_ct_window(ct_image, 120, 80)
bone_window = apply_ct_window(ct_image, 200, 150)

plt.figure(figsize=(16, 10))

plt.subplot(2, 4, 1)
plt.imshow(ct_image, cmap='gray')
plt.title('Original CT Image\n(all values visible)')
plt.axis('off')
plt.colorbar()

plt.subplot(2, 4, 2)
plt.imshow(lung_window, cmap='gray')
plt.title('Lung Window\nC=50, W=100')
plt.axis('off')
plt.colorbar()

plt.subplot(2, 4, 3)
plt.imshow(soft_window, cmap='gray')
plt.title('Soft Tissue Window\nC=120, W=80')
plt.axis('off')
plt.colorbar()

plt.subplot(2, 4, 4)
plt.imshow(bone_window, cmap='gray')
plt.title('Bone Window\nC=200, W=150')
plt.axis('off')
plt.colorbar()

# Show transformation functions
plt.subplot(2, 4, 5)
plt.hist(ct_image.ravel(), bins=256, range=(0, 256), color='gray')
plt.xlabel('Intensity')
plt.title('Histogram')

x = np.arange(256)
for i, (center, width, name, color) in enumerate([
    (50, 100, 'Lung', 'blue'),
    (120, 80, 'Soft', 'green'),
    (200, 150, 'Bone', 'red')
]):
    plt.subplot(2, 4, 6+i)
    low = center - width/2
    high = center + width/2
    y = np.clip((x - low) / (high - low) * 255, 0, 255)
    plt.plot(x, y, color=color, linewidth=2)
    plt.axvline(x=low, color='gray', linestyle='--', alpha=0.5)
    plt.axvline(x=high, color='gray', linestyle='--', alpha=0.5)
    plt.xlabel('Input')
    plt.ylabel('Output')
    plt.title(f'{name} Window')
    plt.xlim([0, 255])
    plt.ylim([0, 260])
    plt.grid(True, alpha=0.3)

plt.suptitle('CT Window/Level Adjustment', fontsize=14)
plt.tight_layout()
plt.show()

## Summary

What we learned:

1. **Min-Max Stretching**: Maps [min, max] to [0, 255]
   $$g = \frac{f - f_{min}}{f_{max} - f_{min}} \times 255$$

2. **Percentile Stretching**: More robust to outliers

3. **Piecewise Linear**: Different mappings for different ranges

4. **Intensity Slicing**: Highlight specific intensity ranges

5. **CT Windowing**: Essential for medical imaging
   - Different windows reveal different tissues
   - Lung, soft tissue, bone windows